In [1]:
import os

import mmdet.datasets
import mmdet.apis
import mmcv

In [2]:
checkpoint_path = "../work_dirs/dh_faster_rcnn_r50_hrnetv2p_1x_coco/epoch_12.pth"
config_path = "../configs/lzy/dh_faster_rcnn_r50_hrnetv2p_1x_coco.py"

cfg = mmcv.Config.fromfile(config_path)
cfg.model.pretrained = None
cfg.data.test.test_mode = True
cfg.test_cfg.max_per_img = 100 # Set to 100 instead of 200 as CustomDataset.evaluate has no maxDets

In [3]:
test_dataset = mmdet.datasets.build_dataset(cfg.data.test)
test_dataloader = mmdet.datasets.build_dataloader(
    test_dataset,
    samples_per_gpu=1,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=False,
    shuffle=False
)

loading annotations into memory...
Done (t=0.10s)
creating index...
index created!


In [4]:
raw_model = mmdet.models.build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)
checkpoint = mmcv.runner.load_checkpoint(raw_model, checkpoint_path, map_location='cpu')
raw_model.CLASSES = test_dataset.CLASSES
model = mmcv.parallel.MMDataParallel(raw_model, device_ids=[0])

In [5]:
results = mmdet.apis.single_gpu_test(model, test_dataloader)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 836/836, 9.3 task/s, elapsed: 90s, ETA:     0s

In [6]:
test_dataset.evaluate(results)


Evaluating bbox...
Loading and preparing results...
DONE (t=0.24s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=2.35s).
Accumulating evaluation results...
DONE (t=0.34s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.356
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.524
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.413
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.373
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.463
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.662
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.711
 Average Recall     (AR) @[ IoU=0.50:0.95 | ar

{'bbox_mAP': 0.356,
 'bbox_mAP_50': 0.524,
 'bbox_mAP_75': 0.413,
 'bbox_mAP_s': -1.0,
 'bbox_mAP_m': -1.0,
 'bbox_mAP_l': 0.373,
 'bbox_mAP_copypaste': '0.356 0.524 0.413 -1.000 -1.000 0.373'}

In [7]:
super(type(test_dataset), test_dataset).evaluate(results, iou_thr=0.5)


+-------+-----+-------+--------+-------+
| class | gts | dets  | recall | ap    |
+-------+-----+-------+--------+-------+
| 0     | 673 | 19102 | 0.973  | 0.742 |
| 1     | 431 | 19021 | 0.940  | 0.308 |
+-------+-----+-------+--------+-------+
| mAP   |     |       |        | 0.525 |
+-------+-----+-------+--------+-------+


{'mAP': 0.5250895023345947}

In [8]:
super(type(test_dataset), test_dataset).evaluate(results, iou_thr=0.8)


+-------+-----+-------+--------+-------+
| class | gts | dets  | recall | ap    |
+-------+-----+-------+--------+-------+
| 0     | 673 | 19102 | 0.820  | 0.527 |
| 1     | 431 | 19021 | 0.652  | 0.150 |
+-------+-----+-------+--------+-------+
| mAP   |     |       |        | 0.339 |
+-------+-----+-------+--------+-------+


{'mAP': 0.33879756927490234}

In [9]:
super(type(test_dataset), test_dataset).evaluate(results, iou_thr=0.3)


+-------+-----+-------+--------+-------+
| class | gts | dets  | recall | ap    |
+-------+-----+-------+--------+-------+
| 0     | 673 | 19102 | 0.982  | 0.764 |
| 1     | 431 | 19021 | 0.961  | 0.339 |
+-------+-----+-------+--------+-------+
| mAP   |     |       |        | 0.552 |
+-------+-----+-------+--------+-------+


{'mAP': 0.5517555475234985}